# Algorithm comparison

Identify the best machine learning algorithm for predicting what category a chemical from the test set belongs to

### Get the data  from [property_calculations](property_calculations.ipynb) and preprocess according to [feature_preprocessing](feature_preprocessing.ipynb)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os.path as path
import pickle

# Load property_array from property_calculations FEMA-JECFA database
BASE_DATA_PATH = path.join(path.expanduser('~'),
                           'Dropbox',
                           'bymt',
                           'data_dumps',
                           'chem_project')

dataset_path = path.join(BASE_DATA_PATH,
                         'property_calculations',
                         'dataset.pkl')

with open(dataset_path, 'rb') as f:
    dataset = pickle.load(f)
    
train_features = dataset['train features']
train_labels = dataset['train labels']

### Test on different classifiers

In [4]:
#Scoring function
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score,\
                            precision_score, recall_score
import scipy.stats as st
import warnings

def t_delta(percent, scores):
    """
    Returns the size of a t-test error bar that defines a given percent confidence interval 
    """
    mean, sem = np.mean(scores), st.sem(scores)
    interval = st.t.interval(percent/float(100), len(scores)-1, loc=mean, scale=sem)
    delta = mean - interval[0]
    return delta


def scores(clf,
           features, labels,
           score_types, 
           title='estimator', printout=True):
    """
    Generates a dictionary with cross_validation scores  and summary statistics
    for a given clf (estimator), features, and labels
    """
    
    if printout:
        print('\n{}:' .format(title))
    
    summary_dict = {title: {}}

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=41)

    for scoring in score_types:
        scores = cross_val_score(clf, features, labels, cv=cv, scoring=scoring)
        mean =  np.mean(scores)
        # t-test 95% confidence interval
        delta = t_delta(95, scores)

        summary_dict[title][scoring] = (mean, delta)

        if printout:
            print('-', scoring, scores)
            print('Mean: {:.2f} (+/-{:.2f})' .format(mean, delta))

    return summary_dict

def multi_tester(step_list, clf_list, score_types):
    """
    Runs scores function on all clfs in clf_list.
    Preappends the steps in step_list to make a pipeline with clf at the end.
    Returns a scores list for each clf and a pipe_dict of all the pipes generated
    """
    pipe_dict = {}
    scores_list = []
    for clf in clf_list:
        new_steps = step_list[:]
        new_steps.append(('estimator', clf))
        new_pipe = Pipeline(new_steps)
        title = str(type(clf))    
        title = title.split('.')[-1][:-2]
        pipe_dict[title] = new_pipe
        scores_list.append(scores(new_pipe, train_features,
                                  train_labels, score_types,
                                  title=title, printout=True))
    return scores_list, pipe_dict

Test on Naive Bayes, Support Vector Machines, AdaBoost, Extra Random Forest, Random Forest, and Logistic Regression classifiers

In [6]:
from sklearn.preprocessing import Imputer, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectPercentile, f_classif

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.pipeline import Pipeline

steps = [('imputer', Imputer(strategy='mean')),
         ('scaler', StandardScaler()),
         ('polynomial', PolynomialFeatures(2, include_bias=False)),
         ('selector', SelectPercentile(f_classif, percentile=10))]

step_list = [('imputer', Imputer(strategy='mean')),
         ('scaler', StandardScaler()),
         ('polynomial', PolynomialFeatures(2, include_bias=False)),
         ('selector', SelectPercentile(f_classif, percentile=10))]

clf_list = [GaussianNB(),
            SVC(),
            AdaBoostClassifier(),
            LogisticRegression(),
            MLPClassifier()]

score_types = ['accuracy', 
               'precision_macro',
               'recall_macro']

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    scores_list, pipe_dict = multi_tester(step_list, clf_list, score_types)


GaussianNB:
- accuracy [ 0.43984221  0.38856016  0.41188119]
Mean: 0.41 (+/-0.06)
- precision_macro [ 0.54971185  0.48648352  0.49605493]
Mean: 0.51 (+/-0.08)
- recall_macro [ 0.6122237   0.50976349  0.54827036]
Mean: 0.56 (+/-0.13)

SVC:
- accuracy [ 0.69822485  0.69033531  0.68316832]
Mean: 0.69 (+/-0.02)
- precision_macro [ 0.63562601  0.44663271  0.58489658]
Mean: 0.56 (+/-0.24)
- recall_macro [ 0.42025539  0.42674731  0.42170998]
Mean: 0.42 (+/-0.01)

AdaBoostClassifier:
- accuracy [ 0.64102564  0.64891519  0.59207921]
Mean: 0.63 (+/-0.08)
- precision_macro [ 0.53800755  0.53786567  0.49579098]
Mean: 0.52 (+/-0.06)
- recall_macro [ 0.50467974  0.47757371  0.51658901]
Mean: 0.50 (+/-0.05)

LogisticRegression:
- accuracy [ 0.66074951  0.65877712  0.63168317]
Mean: 0.65 (+/-0.04)
- precision_macro [ 0.56868878  0.56000533  0.50593773]
Mean: 0.54 (+/-0.08)
- recall_macro [ 0.52186626  0.54045256  0.46772593]
Mean: 0.51 (+/-0.09)

MLPClassifier:
- accuracy [ 0.67455621  0.69625247  0.

### Visualize the different algorithm scores testing

In [7]:
def initialize_plot_dicto(dicto):
    """
    Initialize a flat plot_dicto with keys from dicto and epmty lists
    """
    og_key = list(dicto.keys())[0]
    keys = ['clf'] + list(dicto[og_key].keys())
    plot_dicto = {key: [] for key in keys}
    return plot_dicto

def build_plot_dicto(plot_dicto, scores_list):
    """
    Add values to plot_dicto from scores list
    """
    
    for dicto in scores_list:
        og_key = list(dicto.keys())[0]
        plot_dicto['clf'].append(og_key)
        keys = list(plot_dicto.keys())
        keys.remove('clf')
        for key in keys:
            plot_dicto[key].append(dicto[og_key][key])
    
    return plot_dicto

In [8]:
init_plot_dicto = initialize_plot_dicto(scores_list[0])
plot_dicto = build_plot_dicto(init_plot_dicto, scores_list)
print('plot_dicto = {}' .format(plot_dicto))

plot_dicto = {'precision_macro': [(0.5107500985933533, 0.084658329042625546), (0.55571843544395894, 0.24299057678232139), (0.52388806645439145, 0.060446257788549673), (0.54487727784783979, 0.084463120537438596), (0.5564864206004777, 0.13770794561410526)], 'recall_macro': [(0.55675251566111694, 0.12856423312860116), (0.42290422788325638, 0.0084628313637376551), (0.49961415476620402, 0.049669934604814725), (0.51001491297928025, 0.093860683924294697), (0.52273389578046991, 0.044975608485826268)], 'clf': ['GaussianNB', 'SVC', 'AdaBoostClassifier', 'LogisticRegression', 'MLPClassifier'], 'accuracy': [(0.41342785166090573, 0.063782688252627284), (0.69057615820753682, 0.018708430298698842), (0.62734001210771961, 0.076487825237209983), (0.65040326517858882, 0.040347481955676168), (0.67277781032541129, 0.060643980579775825)]}


In [9]:
clfs = plot_dicto['clf']
score_list = list(plot_dicto.keys())
score_list.remove('clf')
colors = ['#EE3224', '#F78F1E', '#FFC222']

pos = list(range(len(clfs)))
width = 0.25

plt.close('all')
fig, ax = plt.subplots(figsize=(10,7))
for i, score in enumerate(score_list):
    value = plot_dicto[score]
    means, stds = zip(*value)
    plt.bar([p + width*i for p in pos],
            means,
            width,
            yerr=stds,
            alpha=0.5,
            color=colors[i],
            label=score)

ax.set_ylabel('Score')
ax.set_title('Unoptimized algorithm scores')
ax.set_xticks([p + width for p in pos])
ax.set_xticklabels(clfs, rotation=45, horizontalalignment='right')
plt.xlim(min(pos)-width, max(pos)+width*4)

plt.legend(loc='upper right')
plt.grid()
plt.show()

All unoptimized classifiers seem comparable. I will focus on the Support Vector Machine, AdaBoost and Logistic Regression classifiers because they are the most different from eachother.

To find the best one I will exhaustively search all the possible combinations of several hyper-parameters and evaluate them with cross-validation in [parameter_optimization](parameter_optimization.ipynb).